In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
# from skimage.transform import pyramid_gaussian
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import GridSearchCV
# import matplotlib.pyplot as plt
from inspect import signature

# print(glob.glob("../ILIYAN Master Thesis/Dataset/*"))

%matplotlib inline
%precision 2

'%.2f'

In [3]:
df1_t1 = pd.read_csv('data/w2v_fs/df_top_1_stem_1.csv', index_col=0)
df1_t1t2 = pd.read_csv('data/w2v_fs/df_top_1_and_2_stem_1.csv', index_col=0)

df2_t1 = pd.read_csv('data/w2v_fs/df_top_1_stem_1.csv', index_col=0)
df2_t1t2 = pd.read_csv('data/w2v_fs/df_top_1_and_2_stem_1.csv', index_col=0)

In [4]:
df_class = pd.read_csv('data/target.csv', index_col=0)

In [5]:
df1_t1['Class'] = df_class['Class']
df1_t1t2['Class'] = df_class['Class']
df2_t1['Class'] = df_class['Class']
df2_t1t2['Class'] = df_class['Class']

In [6]:
def draw_confusionmatrix(y_test, y_hat):
    plt.figure(figsize=(10,7))
    cm = confusion_matrix(y_test, y_hat)
    ax = sns.heatmap(cm, annot=True, fmt="d")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    acc = accuracy_score(y_test, y_hat)
    print(f"Sum Axis-1 as Classification accuracy: {acc}")
    print('\n')
    print(classification_report(y_test, y_hat))
    print('\n')

In [7]:
def gen_train_and_test(df, test_size=0.20, random_state=42):
    X = df.loc[:, df.columns != 'Class']
    y = df.Class
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test

In [8]:
def split_train_and_test(X, y, test_size=0.20, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)    
    return X_train, X_test, y_train, y_test

In [9]:
def start_model(df, model):
    X_train, X_test, y_train, y_test = gen_train_and_test(df)
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    draw_confusionmatrix(y_test, y_hat)

### GaussianNB

In [10]:
model = GaussianNB()

In [11]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
def run_cross_validation(df, model, scoring='accuracy', cv=10):
    X = df.loc[:, df.columns != 'Class']
    y = df.Class
    
    print("avg accuracy:"+str(np.average(cross_val_score(model, X, y, scoring='accuracy', cv=cv))))
    print("avg f1:"+str(np.average(cross_val_score(model, X, y, scoring='f1', cv=cv))))
    print("avg precision:"+str(np.average(cross_val_score(model, X, y, scoring='precision', cv=cv))))
    print("avg recall:"+str(np.average(cross_val_score(model, X, y, scoring='recall', cv=cv))))
    return None

In [15]:
run_cross_validation(df1_t1, model)

avg accuracy:0.9400000000000001
avg f1:0.9457142857142857
avg precision:0.975
avg recall:0.9333333333333332


In [16]:
run_cross_validation(df1_t1t2, model)

avg accuracy:1.0
avg f1:1.0
avg precision:1.0
avg recall:1.0


In [17]:
run_cross_validation(df2_t1, model)

avg accuracy:0.9400000000000001
avg f1:0.9457142857142857
avg precision:0.975
avg recall:0.9333333333333332


In [18]:
run_cross_validation(df2_t1t2, model)

avg accuracy:1.0
avg f1:1.0
avg precision:1.0
avg recall:1.0


### SVM

In [19]:
def start_grid_cv(df, cv=10, n_jobs=-1):
    param_grid = {
                  'kernel':('linear', 'rbf'),
                  'C':(0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1, 10),
                  'gamma': (0.001, 0.01, 0.1, 1,2,3,'auto'),
                  'decision_function_shape':('ovo','ovr'),
                  'shrinking':(True,False)
                 }
    X_train, X_test, y_train, y_test = gen_train_and_test(df)
    clf = GridSearchCV(estimator=svm.SVC(), param_grid=param_grid, cv=cv, n_jobs=n_jobs)
    clf.fit(X_train, y_train)
    
    print("avg accuracy:"+str(np.average(cross_val_score(clf, X_train, y_train, scoring='accuracy'))))
    print("avg f1:"+str(np.average(cross_val_score(clf, X_train, y_train, scoring='f1'))))
    print("avg precision:"+str(np.average(cross_val_score(clf, X_train, y_train, scoring='precision'))))
    print("avg recall:"+str(np.average(cross_val_score(clf, X_train, y_train, scoring='recall'))))
    
    best_params = clf.best_params_
    print(best_params)
    
    return best_params

In [20]:
params = start_grid_cv(df1_t1, cv=10)

avg accuracy:0.975
avg f1:0.9777777777777779
avg precision:0.96
avg recall:1.0
{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 2, 'kernel': 'rbf', 'shrinking': True}


In [21]:
params = start_grid_cv(df1_t1t2, cv=10)

avg accuracy:0.95
avg f1:0.9555555555555555
avg precision:0.96
avg recall:0.96
{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 1, 'kernel': 'rbf', 'shrinking': True}


In [22]:
params = start_grid_cv(df2_t1, cv=10)

avg accuracy:0.975
avg f1:0.9777777777777779
avg precision:0.96
avg recall:1.0
{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 2, 'kernel': 'rbf', 'shrinking': True}


In [23]:
params = start_grid_cv(df2_t1t2, cv=10)

avg accuracy:0.95
avg f1:0.9555555555555555
avg precision:0.96
avg recall:0.96
{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 1, 'kernel': 'rbf', 'shrinking': True}
